# Results from the HTRC Harvard Collection

We ran the two extraction runs - total counts for all tokens, and counts for tokens that appear on the same pages with "literature" - on the 820k Harvard volumes in the HTRC data.

~43.5 billion total observed tokens:

In [1]:
from hol.queries import CountQueries

cq = CountQueries()

print(cq.total_token_count())

43401065888


9547 unique tokens:

In [2]:
tokens = cq.tokens()
print(len(tokens))

9547
